In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as npimg 
import numpy as np
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2

In [ ]:
def YellowAndWhite(hsv_image):
    masks = []
    low_white,high_white = np.array([0, 0, 200]),np.array([180, 25, 255])
    low_yellow,high_yellow = np.array([15, 40, 205]),np.array([25, 255, 255])
    white = cv2.inRange(hsv_image, low_white, high_white)
    masks.append(white)
    yellow = cv2.inRange(hsv_image, low_yellow, high_yellow)
    masks.append(yellow)
    return cv2.add(*masks)

In [ ]:
def Force3D(mask, image):
    masked_image = np.zeros_like(image)
    for i in range(3): 
        masked_image[:,:,i] = mask.copy()
    return masked_image

In [ ]:
def AutoCanny(image):   #new
    v = np.median(image)
    
    low = int(max(0, v*.77))
    high = int(min(255, v*1.33))
    
    edged_image = cv2.Canny(image, low, high)
    return edged_image

In [ ]:
def RegionOfInterest(image, points): 
    mask = np.zeros_like(image)   
    ignore_color = 255

    cv2.fillPoly(mask, [points],ignore_color)
    masked_image = cv2.bitwise_and(image, mask)
    
    return masked_image

In [ ]:
def ProcessImage(image):
    
    width,height = len(image[0]),len(image)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    color_mask = YellowAndWhite(hsv_image)

    masked_image = Force3D(color_mask, image) 

    edge_mask = AutoCanny(masked_image)   #new
      
    edged_image = Force3D(edge_mask, image) 
    
    p1,p2,p3,p4  = [10,height-10],[int(29/60*width),int(9/16*height)],[int(31/60*width),int(9/16*height)],[width-10,height-10]
    points = np.array([p1,p2,p3,p4], dtype=np.int32) 
    region_image= RegionOfInterest(edged_image, points) 
    
    grey_image = cv2.cvtColor(region_image, cv2.COLOR_RGB2GRAY)
    
    return grey_image

In [ ]:
def ImageToLines(image): 
    rho = 2
    theta = np.pi/180  #modified values.
    threshold = 60
    min_line_length = 70
    max_line_gap = 15
    
    return cv2.HoughLinesP(image, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
    

In [ ]:
def HoughlinesToLanelines(lines = []):  #new
    global left_lane_line, right_lane_line
    
    if lines != []:
        for line in lines:
            for x1,y1,x2,y2 in line:
                if x1>width/2 and x2>width/2:
                    if (y2-y1)/(x2-x1) > 0.4: 
                        right_lane_line.append([x1,y1])
                        right_lane_line.append([x2,y2])
                elif x1<width/2 and x2<width/2:
                    if (y2-y1)/(x2-x1) < -0.4: 
                        left_lane_line.append([x1,y1])
                        left_lane_line.append([x2,y2])        

        while len(left_lane_line)> 50:
            del left_lane_line[0]
        while len(right_lane_line)> 50:
            del right_lane_line[0]
    
    leftx,lefty = [point[0] for point in left_lane_line],[point[1] for point in left_lane_line]
    rightx,righty = [point[0] for point in right_lane_line],[point[1] for point in right_lane_line]

    return leftx,lefty,rightx,righty

In [ ]:
def FofY(y,mxb):
    return (y-mxb[1])/mxb[0]

In [ ]:
global left_lane_line, right_lane_line
left_lane_line, right_lane_line = [],[]

def LaneLineDetector(original_image):
    
    image = np.copy(original_image)
    global width,height
    height,width = image.shape[0],image.shape[1]
    
    processed_image = ProcessImage(image)

    lines = ImageToLines(processed_image) #new
    
    leftx,lefty,rightx,righty = HoughlinesToLanelines(lines)
    
    rightmxb = np.polyfit(rightx,righty,1)
    leftmxb = np.polyfit(leftx,lefty,1)
    
    fl = np.poly1d(leftmxb)
    fr = np.poly1d(rightmxb)
    
    
    horizon = height/1.65
    lx1,lx2,rx1,rx2 = int(min(leftx)),int(FofY(horizon,leftmxb)),int(FofY(horizon,rightmxb)),int(max(rightx))
    ly1,ly2,ry1,ry2 = int(fl(lx1)),int(fl(lx2)),int(fr(rx1)),int(fr(rx2))
    
    lined_image = np.zeros_like(image) 
    cv2.line(lined_image, (lx1,ly1), (lx2, ly2), [0,0,255], 5) 
    cv2.line(lined_image, (rx1,ry1), (rx2, ry2), [0,0,255], 5)
    

    laned_image = cv2.addWeighted(original_image, 0.8, lined_image, 1, 0) 

    return laned_image 

# Tests

## Yellow line video

In [ ]:
yellow_clip_output = 'solidYellowLeft1.mp4'
clip2 = VideoFileClip("MathMagicMachine/TestVideos/solidYellowLeft.mp4")
yellow_clip = clip2.fl_image(LaneLineDetector)
%time yellow_clip.write_videofile(yellow_clip_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(yellow_clip_output))

## Challenge video

In [ ]:
final_clip_output = 'challenge2.mp4'
clip = VideoFileClip("MathMagicMachine/TestVideos/challenge.mp4")
final_clip = clip.fl_image(LaneLineDetector)
%time final_clip.write_videofile(final_clip_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(final_clip_output))

## White line video

In [ ]:
white_clip_output = 'solidWhiteRight1.mp4'
clip1 = VideoFileClip("TestVideos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(LaneLineDetector)
%time white_clip.write_videofile(white_clip_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(white_clip_output))